In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

C:\Users\liuliyian\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting .\train-images-idx3-ubyte.gz
Extracting .\train-labels-idx1-ubyte.gz
Extracting .\t10k-images-idx3-ubyte.gz
Extracting .\t10k-labels-idx1-ubyte.gz


In [3]:
train_features = mnist.train.images
valid_features = mnist.validation.images
test_features = mnist.test.images
print(train_features.shape)
print(valid_features.shape)
print(test_features.shape)
print(type(train_features))

(55000, 28, 28, 1)
(5000, 28, 28, 1)
(10000, 28, 28, 1)
<class 'numpy.ndarray'>


In [5]:
# Parameters
learning_rate = 0.00001
epochs = 10
batch_size = 128

# Number of samples to calculate validation and accuracy
# Decrease this if you're running out of memory to calculate accuracy
test_valid_size = 256

# Network Parameters
n_classes = 10  # MNIST total classes (0-9 digits)
dropout = 0.75  # Dropout, probability to keep units

# Store layers weight & bias
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))}

def conv2d(x, W, b, strides = 1):
    x = tf.nn.conv2d(x, W, strides = [1, strides, strides, 1], padding = 'SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k = 2):
    return tf.nn.max_pool(x, ksize = [1,k,k,1], strides = [1,k,k,1], padding='SAME')

def conv_net(x, weights, biases, dropout):
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k = 2)
    
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k = 2)
    
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

# tf Graph input
x = tf.placeholder(tf.float32, [None, 28,28,1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

#Model
logits = conv_net(x, weights, biases, keep_prob)

#Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.
            })
            print('Epoch {:>2}, Batch {:>3} -'
                  'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))
            
     # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))        

Epoch  1, Batch   1 -Loss: 59110.7383 Validation Accuracy: 0.113281
Epoch  1, Batch   2 -Loss: 53205.4062 Validation Accuracy: 0.125000
Epoch  1, Batch   3 -Loss: 45097.4375 Validation Accuracy: 0.132812
Epoch  1, Batch   4 -Loss: 37744.7734 Validation Accuracy: 0.128906
Epoch  1, Batch   5 -Loss: 32006.0977 Validation Accuracy: 0.136719
Epoch  1, Batch   6 -Loss: 31654.5742 Validation Accuracy: 0.121094
Epoch  1, Batch   7 -Loss: 25265.4570 Validation Accuracy: 0.128906
Epoch  1, Batch   8 -Loss: 23838.8789 Validation Accuracy: 0.144531
Epoch  1, Batch   9 -Loss: 24121.1055 Validation Accuracy: 0.144531
Epoch  1, Batch  10 -Loss: 22720.9062 Validation Accuracy: 0.160156
Epoch  1, Batch  11 -Loss: 18704.6387 Validation Accuracy: 0.195312
Epoch  1, Batch  12 -Loss: 21633.6855 Validation Accuracy: 0.191406
Epoch  1, Batch  13 -Loss: 18599.5117 Validation Accuracy: 0.210938
Epoch  1, Batch  14 -Loss: 19812.4023 Validation Accuracy: 0.214844
Epoch  1, Batch  15 -Loss: 17922.0352 Validation

Epoch  1, Batch 122 -Loss:  3762.9170 Validation Accuracy: 0.648438
Epoch  1, Batch 123 -Loss:  3455.6191 Validation Accuracy: 0.636719
Epoch  1, Batch 124 -Loss:  4268.6719 Validation Accuracy: 0.636719
Epoch  1, Batch 125 -Loss:  5452.7544 Validation Accuracy: 0.656250
Epoch  1, Batch 126 -Loss:  3215.1860 Validation Accuracy: 0.644531
Epoch  1, Batch 127 -Loss:  5262.7373 Validation Accuracy: 0.648438
Epoch  1, Batch 128 -Loss:  3903.0293 Validation Accuracy: 0.652344
Epoch  1, Batch 129 -Loss:  4761.1104 Validation Accuracy: 0.652344
Epoch  1, Batch 130 -Loss:  4140.8203 Validation Accuracy: 0.656250
Epoch  1, Batch 131 -Loss:  3678.5962 Validation Accuracy: 0.656250
Epoch  1, Batch 132 -Loss:  3238.6118 Validation Accuracy: 0.664062
Epoch  1, Batch 133 -Loss:  5091.3838 Validation Accuracy: 0.656250
Epoch  1, Batch 134 -Loss:  4315.3496 Validation Accuracy: 0.664062
Epoch  1, Batch 135 -Loss:  4863.1572 Validation Accuracy: 0.664062
Epoch  1, Batch 136 -Loss:  5133.1318 Validation

Epoch  1, Batch 243 -Loss:  2289.3623 Validation Accuracy: 0.710938
Epoch  1, Batch 244 -Loss:  2341.2222 Validation Accuracy: 0.703125
Epoch  1, Batch 245 -Loss:  2539.7456 Validation Accuracy: 0.703125
Epoch  1, Batch 246 -Loss:  2143.1948 Validation Accuracy: 0.699219
Epoch  1, Batch 247 -Loss:  3858.4856 Validation Accuracy: 0.707031
Epoch  1, Batch 248 -Loss:  2333.3784 Validation Accuracy: 0.707031
Epoch  1, Batch 249 -Loss:  2143.6885 Validation Accuracy: 0.707031
Epoch  1, Batch 250 -Loss:  2398.7102 Validation Accuracy: 0.699219
Epoch  1, Batch 251 -Loss:  2530.5178 Validation Accuracy: 0.695312
Epoch  1, Batch 252 -Loss:  2253.4116 Validation Accuracy: 0.703125
Epoch  1, Batch 253 -Loss:  2737.0627 Validation Accuracy: 0.703125
Epoch  1, Batch 254 -Loss:  2389.4268 Validation Accuracy: 0.699219
Epoch  1, Batch 255 -Loss:  2959.9395 Validation Accuracy: 0.687500
Epoch  1, Batch 256 -Loss:  3014.4785 Validation Accuracy: 0.687500
Epoch  1, Batch 257 -Loss:  2502.5422 Validation

Epoch  1, Batch 364 -Loss:  1850.3451 Validation Accuracy: 0.734375
Epoch  1, Batch 365 -Loss:  1807.1587 Validation Accuracy: 0.734375
Epoch  1, Batch 366 -Loss:  1304.2604 Validation Accuracy: 0.746094
Epoch  1, Batch 367 -Loss:  2338.1028 Validation Accuracy: 0.742188
Epoch  1, Batch 368 -Loss:  1760.8284 Validation Accuracy: 0.742188
Epoch  1, Batch 369 -Loss:  1905.3514 Validation Accuracy: 0.750000
Epoch  1, Batch 370 -Loss:  1599.5576 Validation Accuracy: 0.750000
Epoch  1, Batch 371 -Loss:  1575.7805 Validation Accuracy: 0.750000
Epoch  1, Batch 372 -Loss:  1366.8708 Validation Accuracy: 0.746094
Epoch  1, Batch 373 -Loss:  1208.5664 Validation Accuracy: 0.750000
Epoch  1, Batch 374 -Loss:  1806.9033 Validation Accuracy: 0.734375
Epoch  1, Batch 375 -Loss:  1742.5389 Validation Accuracy: 0.746094
Epoch  1, Batch 376 -Loss:  1884.9889 Validation Accuracy: 0.750000
Epoch  1, Batch 377 -Loss:  1295.5885 Validation Accuracy: 0.742188
Epoch  1, Batch 378 -Loss:  2224.4922 Validation

Epoch  2, Batch  56 -Loss:  1043.3977 Validation Accuracy: 0.773438
Epoch  2, Batch  57 -Loss:  1098.0969 Validation Accuracy: 0.769531
Epoch  2, Batch  58 -Loss:  1485.9336 Validation Accuracy: 0.765625
Epoch  2, Batch  59 -Loss:  1310.0795 Validation Accuracy: 0.765625
Epoch  2, Batch  60 -Loss:  1225.2894 Validation Accuracy: 0.769531
Epoch  2, Batch  61 -Loss:  1375.2190 Validation Accuracy: 0.765625
Epoch  2, Batch  62 -Loss:  1510.9191 Validation Accuracy: 0.757812
Epoch  2, Batch  63 -Loss:  1412.8418 Validation Accuracy: 0.761719
Epoch  2, Batch  64 -Loss:  1049.7141 Validation Accuracy: 0.761719
Epoch  2, Batch  65 -Loss:  2047.5714 Validation Accuracy: 0.769531
Epoch  2, Batch  66 -Loss:  1606.4467 Validation Accuracy: 0.773438
Epoch  2, Batch  67 -Loss:  1921.2140 Validation Accuracy: 0.769531
Epoch  2, Batch  68 -Loss:  1591.1808 Validation Accuracy: 0.757812
Epoch  2, Batch  69 -Loss:  1388.2535 Validation Accuracy: 0.765625
Epoch  2, Batch  70 -Loss:   990.6683 Validation

...

Epoch  9, Batch 320 -Loss:   243.6740 Validation Accuracy: 0.882812
Epoch  9, Batch 321 -Loss:   293.0971 Validation Accuracy: 0.878906
Epoch  9, Batch 322 -Loss:   351.5507 Validation Accuracy: 0.878906
Epoch  9, Batch 323 -Loss:   274.2908 Validation Accuracy: 0.878906
Epoch  9, Batch 324 -Loss:   329.3110 Validation Accuracy: 0.875000
Epoch  9, Batch 325 -Loss:   563.7305 Validation Accuracy: 0.875000
Epoch  9, Batch 326 -Loss:   190.6845 Validation Accuracy: 0.875000
Epoch  9, Batch 327 -Loss:   143.6575 Validation Accuracy: 0.875000
Epoch  9, Batch 328 -Loss:   209.2398 Validation Accuracy: 0.875000
Epoch  9, Batch 329 -Loss:   232.4567 Validation Accuracy: 0.871094
Epoch  9, Batch 330 -Loss:   270.9940 Validation Accuracy: 0.871094
Epoch  9, Batch 331 -Loss:   197.6400 Validation Accuracy: 0.867188
Epoch  9, Batch 332 -Loss:   242.9366 Validation Accuracy: 0.871094
Epoch  9, Batch 333 -Loss:   324.5681 Validation Accuracy: 0.867188
Epoch  9, Batch 334 -Loss:   341.3190 Validation

Epoch 10, Batch  12 -Loss:   198.3803 Validation Accuracy: 0.882812
Epoch 10, Batch  13 -Loss:   137.2815 Validation Accuracy: 0.882812
Epoch 10, Batch  14 -Loss:   378.6146 Validation Accuracy: 0.882812
Epoch 10, Batch  15 -Loss:   270.0219 Validation Accuracy: 0.882812
Epoch 10, Batch  16 -Loss:   350.1678 Validation Accuracy: 0.878906
Epoch 10, Batch  17 -Loss:   307.5413 Validation Accuracy: 0.878906
Epoch 10, Batch  18 -Loss:   352.1582 Validation Accuracy: 0.878906
Epoch 10, Batch  19 -Loss:   286.7087 Validation Accuracy: 0.882812
Epoch 10, Batch  20 -Loss:   209.2397 Validation Accuracy: 0.882812
Epoch 10, Batch  21 -Loss:   279.8946 Validation Accuracy: 0.882812
Epoch 10, Batch  22 -Loss:   249.8125 Validation Accuracy: 0.882812
Epoch 10, Batch  23 -Loss:   302.4532 Validation Accuracy: 0.882812
Epoch 10, Batch  24 -Loss:   367.6737 Validation Accuracy: 0.886719
Epoch 10, Batch  25 -Loss:   317.1382 Validation Accuracy: 0.882812
Epoch 10, Batch  26 -Loss:   415.5106 Validation

Epoch 10, Batch 133 -Loss:   331.5616 Validation Accuracy: 0.878906
Epoch 10, Batch 134 -Loss:   182.5441 Validation Accuracy: 0.878906
Epoch 10, Batch 135 -Loss:   107.4515 Validation Accuracy: 0.878906
Epoch 10, Batch 136 -Loss:   283.5916 Validation Accuracy: 0.878906
Epoch 10, Batch 137 -Loss:   259.0297 Validation Accuracy: 0.878906
Epoch 10, Batch 138 -Loss:   263.1772 Validation Accuracy: 0.875000
Epoch 10, Batch 139 -Loss:   240.6015 Validation Accuracy: 0.878906
Epoch 10, Batch 140 -Loss:   275.1524 Validation Accuracy: 0.878906
Epoch 10, Batch 141 -Loss:   261.7658 Validation Accuracy: 0.882812
Epoch 10, Batch 142 -Loss:   217.4961 Validation Accuracy: 0.878906
Epoch 10, Batch 143 -Loss:   220.2959 Validation Accuracy: 0.882812
Epoch 10, Batch 144 -Loss:   333.3273 Validation Accuracy: 0.882812
Epoch 10, Batch 145 -Loss:   194.2184 Validation Accuracy: 0.882812
Epoch 10, Batch 146 -Loss:   311.0861 Validation Accuracy: 0.882812
Epoch 10, Batch 147 -Loss:   322.4970 Validation

Epoch 10, Batch 254 -Loss:   243.8792 Validation Accuracy: 0.882812
Epoch 10, Batch 255 -Loss:   229.7062 Validation Accuracy: 0.886719
Epoch 10, Batch 256 -Loss:   275.2100 Validation Accuracy: 0.890625
Epoch 10, Batch 257 -Loss:   300.0392 Validation Accuracy: 0.886719
Epoch 10, Batch 258 -Loss:   277.1902 Validation Accuracy: 0.890625
Epoch 10, Batch 259 -Loss:   293.2137 Validation Accuracy: 0.886719
Epoch 10, Batch 260 -Loss:   274.7423 Validation Accuracy: 0.882812
Epoch 10, Batch 261 -Loss:   345.1918 Validation Accuracy: 0.882812
Epoch 10, Batch 262 -Loss:   256.2610 Validation Accuracy: 0.882812
Epoch 10, Batch 263 -Loss:   160.3600 Validation Accuracy: 0.882812
Epoch 10, Batch 264 -Loss:   240.1414 Validation Accuracy: 0.882812
Epoch 10, Batch 265 -Loss:   155.2591 Validation Accuracy: 0.882812
Epoch 10, Batch 266 -Loss:   319.8223 Validation Accuracy: 0.882812
Epoch 10, Batch 267 -Loss:   303.5159 Validation Accuracy: 0.882812
Epoch 10, Batch 268 -Loss:   266.3799 Validation

Epoch 10, Batch 375 -Loss:   250.8849 Validation Accuracy: 0.886719
Epoch 10, Batch 376 -Loss:   360.2244 Validation Accuracy: 0.886719
Epoch 10, Batch 377 -Loss:    62.4511 Validation Accuracy: 0.886719
Epoch 10, Batch 378 -Loss:   238.5804 Validation Accuracy: 0.890625
Epoch 10, Batch 379 -Loss:   290.3075 Validation Accuracy: 0.890625
Epoch 10, Batch 380 -Loss:   245.8355 Validation Accuracy: 0.890625
Epoch 10, Batch 381 -Loss:   167.4676 Validation Accuracy: 0.890625
Epoch 10, Batch 382 -Loss:   299.8325 Validation Accuracy: 0.890625
Epoch 10, Batch 383 -Loss:   273.7503 Validation Accuracy: 0.890625
Epoch 10, Batch 384 -Loss:   369.3250 Validation Accuracy: 0.890625
Epoch 10, Batch 385 -Loss:   235.7878 Validation Accuracy: 0.890625
Epoch 10, Batch 386 -Loss:   247.6782 Validation Accuracy: 0.890625
Epoch 10, Batch 387 -Loss:   355.7725 Validation Accuracy: 0.890625
Epoch 10, Batch 388 -Loss:   167.0888 Validation Accuracy: 0.886719
Epoch 10, Batch 389 -Loss:   246.6230 Validation

In [22]:
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
conv1 = conv2d(x, weights['wc1'], biases['bc1'])
print(conv1)
conv1 = maxpool2d(conv1, k = 2)
print(conv1),print()

conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
print(conv2)
conv2 = maxpool2d(conv2, k = 2)
print(conv2),print()

fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
print(fc1)
fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
print(fc1)
fc1 = tf.nn.relu(fc1)
fc1 = tf.nn.dropout(fc1, dropout)
print(fc1),print()
out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
print(out)

Tensor("Relu_20:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("MaxPool_13:0", shape=(?, 14, 14, 32), dtype=float32)

Tensor("Relu_21:0", shape=(?, 14, 14, 64), dtype=float32)
Tensor("MaxPool_14:0", shape=(?, 7, 7, 64), dtype=float32)

Tensor("Reshape_4:0", shape=(?, 3136), dtype=float32)
Tensor("Add_3:0", shape=(?, 1024), dtype=float32)
Tensor("dropout_2/mul:0", shape=(?, 1024), dtype=float32)

Tensor("Add_4:0", shape=(?, 10), dtype=float32)
